In [ ]:
import os
import sys
import pickle


In [ ]:
'''
先尝试纯文本化方法
'''
WIKIPEDIA_CONCEPT_STORE_PATH = 'C:/workspace/SOworkspace/data/wikipedia_concepts/text_content/'
stemmed_wiki_descriptions_path = os.path.join(WIKIPEDIA_CONCEPT_STORE_PATH, 'stemmed/stemmed.txt')
#先把所有wikipedia的概念存起来
pure_wiki_concept_word_path = os.path.join(WIKIPEDIA_CONCEPT_STORE_PATH, 'wikipedia_concept_words.pkl')


#这个cell别跑了，在nb里跑太慢，我放在py里跑了
"""
with open(stemmed_wiki_descriptions_path, 'r', encoding='utf-8') as rf, open(pure_wiki_concept_word_path, 'wb') as wf:
    wiki_concept_words = []
    while True:
        text_line = rf.readline()
        if text_line:
            concept_word = str(text_line).split('|||')[0].strip()
            wiki_concept_words.append(concept_word)
            print("\r",len(wiki_concept_words), concept_word,end="",flush=True)
        else:
            break
    pickle.dump(wiki_concept_words, wf)
"""

In [ ]:
with open(pure_wiki_concept_word_path, 'rb') as rf:
    wiki_concept_words = pickle.load(rf)
    len(wiki_concept_words)

In [ ]:
wiki_concept_words[:10]

In [ ]:
javadoc_description_path = 'C:/workspace/SOworkspace/data/apidoc_description/javadoc_descriptions.pkl'
with open(javadoc_description_path, 'rb') as rf:
    javadoc_description = pickle.load(rf)

'''
这个本机跑太慢了，就放到服务器上跑了。。
跑完的文件放在了C:\workspace\SOworkspace\data\wikipedia_concepts\text_content\javadoc_wiki_concepts_pure_text.pkl文件夹里

javadoc_wiki_concept_path = 'C:/workspace/SOworkspace/data/apidoc_description/javadoc_wiki_concepts_pure_text.pkl'
from tqdm import tqdm
javadoc_concepts = {}
for concept, description in tqdm(javadoc_description.items()):
    temp_words = []
    for word in wiki_concept_words:
        if word in description:
            temp_words.append(word)
    javadoc_concepts[concept] = temp_words
with open(javadoc_wiki_concept_path, 'wb') as wf:
    pickle.dump(javadoc_concepts, wf)
'''

In [ ]:
#好，看看跑出来的直接在description里识别文本的识别结果怎么样
with open(r'C:\workspace\SOworkspace\data\wikipedia_concepts\text_content\javadoc_wiki_concepts_pure_text.pkl', 'rb') as rf:
    javadocdesc_wikiconcept_puretext_match = pickle.load(rf)
javadocdesc_wikiconcept_puretext_match_list = list(javadocdesc_wikiconcept_puretext_match.items())

javadocdesc_wikiconcept_puretext_match_list[12345]

In [ ]:
javadocdesc_wikiconcept_puretext_match_list = list(javadocdesc_wikiconcept_puretext_match.items())

In [ ]:
#去掉空格
desc_wiki_text_match_list_non_white = [(k, [i for i in v if len(i) > 2]) for k, v in javadocdesc_wikiconcept_puretext_match_list]
desc_wiki_text_match_list_non_white[13457]

In [ ]:
'''试着算一下tf-idf提取词频比较高的词汇'''
from gensim import corpora
#生成api和描述分别的列表，用描述列表做tf-idf训练。到时候再和api列表做对应
api_names = []
descs = []
for k, v in desc_wiki_text_match_list_non_white:
    api_names.append(k)
    descs.append(v)
descs[0: 5]

In [ ]:
dictionay = corpora.Dictionary(descs)
desc_corpus = [dictionay.doc2bow(text) for text in descs]
desc_corpus[0]

In [ ]:
from gensim import models
tfidf = models.TfidfModel(desc_corpus)
tfidf.save(r'C:\workspace\SOworkspace\data\cache\javadoc_description_wikiwords.tfidf.model')

In [ ]:
word2id = dictionay.token2id
id2word = dict([(v, k) for k, v in word2id.items()])
id2word

In [31]:
def get_tfidf_desc(desc: list):
    bow = dictionay.doc2bow(desc)
    desc_tf_idf = tfidf[bow]
    return sorted([(id2word[id], vec) for id, vec in desc_tf_idf], key=lambda v : v[1], reverse=True)

def show_desc_tfidf_status(desc_id):
    print(api_names[desc_id])
    print('=====')
    print(get_tfidf_desc(descs[desc_id][0:30]))

show_desc_tfidf_status(12313)

api/java.base/java/util/Calendar.html#NOVEMBER
=====
[('ego', 0.41907358702457753), ('mon', 0.32930740796692987), ('event', 0.29366648096222003), ('even', 0.27753431805579765), ('oria', 0.2479148020387051), ('calendar', 0.242685009766123), ('mont', 0.2189550797563686), ('month', 0.2189550797563686), ('year', 0.21218309183773393), ('leve', 0.2110005856434952), ('gor', 0.17608155440633194), ('dar', 0.1665797765032846), ('nda', 0.16039185782601778), ('field', 0.1588381133489189), ('ear', 0.15544240911602447), ('indic', 0.1549156186120962), ('dic', 0.14225174556457393), ('indi', 0.14155394243509964), ('eve', 0.10293486151870439), ('ele', 0.10235513085929446), ('tin', 0.08695521896887032), ('cat', 0.08415072835706143), ('cal', 0.07987346474800702), ('ati', 0.0516091753409838), ('and', 0.04933550084137336), ('ing', 0.03937574756681835)]
